## imports

In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2025-05-27 15:21:19.851071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-27 15:21:28.109554: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-27 15:22:00.500780: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import pickle
import sys
import os
from tqdm import tqdm
import numpy as np
sys.path.append('../')
sys.path.append('../../../../')

## constants

In [3]:
data_for_training_dir_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training'
source_dataset_dir_path = f'{data_for_training_dir_path}/03_04_with_pesto'
target_dataset_dir_path = f'{data_for_training_dir_path}/03_04_with_pesto_and_coord'

MAX_NUMBER_OF_COMPONENTS = 10

## utils

In [4]:
# copied from data_development_utils.py
# modified a bit
# TODO: the main risk here: sorted might change order from original creation 

def extract_protein_data(proteins, max_number_of_components):
    residue_indexes = []
    for protein in proteins:
        # Sort components by average_ubiq in descending order and take the top 10
        top_components = sorted(protein.connected_components_tuples, key=lambda x: x[1]['average_scanNet_ubiq'], reverse=True)[
                         :max_number_of_components]
        residue_indexes.append([component[2] for component in top_components])
    return residue_indexes

## load data from source

for fold_index in range(5) <br>
for set_name in ['train', 'validation', 'test'] <br>

Currently folds_training_dicts[fold_index] of set_name, contains:
- sizes (will be broadcasted)
- num_patches (will be broadcasted)
- components (the features matrix, of shape [protein_amount, 10, 9] -> 10=patches_amount, 9=2 for scannet, 5 for pesto, 1 for plddt, 1 for size)
- uniprots
- labels

We want to add one more matrix (again, per fold_index, per set_name). <br>
This matrix will be of shape [protein_amount, 10, 3] -> x, y, z coordiantes. <br>
x, y, z coordinates will be the average of the ca coords in the patch. <br>
the new key will be:

- coordinates

In [6]:
with open(f'{source_dataset_dir_path}/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-05-26 08:00:43.328209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# TODO: itterate in for loop
fold_index = 0
fold = folds_training_dicts[fold_index]
[x for x in fold.keys() if 'train' in x]

['sizes_train',
 'components_train',
 'num_patches_train',
 'uniprots_train',
 'labels_train']

In [8]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/patches_dicts/03_04_with_pesto/merged_protein_objects.pkl', 'rb') as f:
# with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/patches_dicts/03_04_with_pesto/proteinObjectsWithEvoluion0', 'rb') as f:
    merged_protein_objects = pickle.load(f)

/home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [9]:
set_name = 'train'  # TODO: iterate in for loop
uniprots = set(fold[f'uniprots_{set_name}'])
len(uniprots), list(uniprots)[0]

(76632, 'A0A7N9D9P4')

In [10]:
fold[f'components_{set_name}'].shape

TensorShape([76632, 10, 9])

In [ ]:
data_set_res_indexes = {k: extract_protein_data([v], MAX_NUMBER_OF_COMPONENTS)[0] for k, v in merged_protein_objects.items()}
# TODO: remove
data_set_res_indexes = {k: v for k, v in data_set_res_indexes.items() if 'J' == k[0]}
len(data_set_res_indexes)

171

## add coord

In [32]:
# TODO
# we have the residue indexes in data_set_res_indexes
# now we just need to fetch the ca coord of each residue, then average accross the inner lists 

In [43]:
max({k: len(v) for k, v in data_set_res_indexes.items()}.values())

10

In [68]:
data_set_res_indexes

{'J9NS22': [[118], [35, 37, 16, 48, 18, 50, 21]],
 'J9NTA2': [[15,
   16,
   17,
   18,
   19,
   21,
   33,
   35,
   37,
   40,
   41,
   48,
   50,
   84,
   103,
   104,
   107,
   110,
   111,
   113,
   115,
   116,
   117,
   118],
  [91]],
 'J9NV53': [[165, 167, 169, 170, 138, 171, 172]],
 'J9NWU7': [[98, 122, 102, 107, 110, 49, 113, 118, 121, 58, 57],
  [73, 75, 78, 81, 85, 89],
  [6]],
 'J9NX45': [[102,
   105,
   106,
   41,
   108,
   109,
   110,
   42,
   112,
   113,
   115,
   20,
   86,
   87,
   118,
   120]],
 'J9NYP5': [[0,
   1,
   3,
   4,
   10,
   11,
   12,
   13,
   14,
   39,
   41,
   42,
   43,
   44,
   46,
   47,
   48,
   56,
   58,
   59,
   62,
   63,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   73,
   74,
   75,
   76,
   84,
   87,
   88],
  [124]],
 'J9P0N9': [[128,
   131,
   132,
   134,
   136,
   137,
   138,
   139,
   141,
   25,
   86,
   92,
   94,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   110,
   111,
   117,
   120,

In [46]:
# lets try to get the ca coord of a single residue, from a protein object:
# TODO: this should iterate

data_set_coordinates = dict()


for uniprot_id, protein_object in tqdm({k: v for k, v in merged_protein_objects.items() if k in data_set_res_indexes}.items()):
    data_set_coordinates[uniprot_id] = []
    structure = protein_object.get_structure()
    model = structure.child_list[0]
    assert len(model) == 1
    protein_indexes = data_set_res_indexes[uniprot_id]
    
    for i, patch_indexes in enumerate(protein_indexes):
        data_set_coordinates[uniprot_id].append([])
        for residue_index in patch_indexes:
            for chain in model:
                coord = chain.child_list[residue_index]['CA'].coord  # TODO: is this logic correct?
                data_set_coordinates[uniprot_id][i].append(coord)
    
        if len(data_set_coordinates[uniprot_id][i]) == 0:
            data_set_coordinates[uniprot_id][i] = None
        else:
            data_set_coordinates[uniprot_id][i] = np.vstack(data_set_coordinates[uniprot_id][i])


100%|██████████| 171/171 [00:16<00:00, 10.18it/s]


## check validity

In [4]:
with open(f'{target_dataset_dir_path}/all_patches_ca_coordinates.pkl', 'rb') as f:
    all_patches_ca_coordinates = pickle.load(f)

with open(f'{target_dataset_dir_path}/data_set_res_indexes.pkl', 'rb') as f:
    data_set_res_indexes = pickle.load(f)    

In [12]:
all_patches_ca_coordinates

{'A0A022QL98': [array([[  7.224,  15.01 , -29.285],
         [  7.596,  15.595, -32.995],
         [  7.363,  12.279, -34.892],
         [  7.876,   8.014, -40.137],
         [  5.806,  16.993, -26.353]], dtype=float32),
  array([[ -3.141,   8.278,  16.4  ],
         [ -4.457,  10.937,  18.841],
         [ -7.498,  10.695,  18.587],
         [ -9.644,  11.068,  15.405],
         [ -8.425,  11.226,  11.772],
         [-10.518,  11.743,   8.628],
         [-14.548,  13.885,  22.479],
         [-15.629,  18.807,  24.074],
         [-15.489,  19.727,  20.349],
         [-11.077,  24.221,  16.92 ],
         [-13.887,  20.745,   8.755],
         [  8.15 ,   3.368,  -2.161],
         [ 11.457,   3.166,   3.548],
         [  5.667,   7.345,  -0.581],
         [  3.881,   9.164,  -6.556],
         [  0.783,   8.519,  12.62 ],
         [  1.188,  11.358,  10.042],
         [  1.202,  14.597,   4.058],
         [  0.864,  17.38 ,  -2.007],
         [  5.139,  18.765,  -9.72 ]], dtype=float32),
  

## merge with folds

In [37]:
with open(f'{source_dataset_dir_path}/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-05-27 16:02:08.327323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
patches_ca_avg_coordinates = {uniprot_id: [np.average(c, axis=0) if c is not None else None for c in coordinates] for uniprot_id, coordinates in all_patches_ca_coordinates.items()}
{k: len(v) for k, v in patches_ca_avg_coordinates.items()}
patches_ca_avg_coordinates = {k: np.vstack(v) if (len(v) > 0) else None for k, v in patches_ca_avg_coordinates.items()}

In [43]:
fold_index = 0
fold = folds_training_dicts[fold_index]
set_name = 'train'
uniprots = fold[f'uniprots_{set_name}']
uniprots = [u for u in uniprots if u in patches_ca_avg_coordinates]  # TODO: remove this line

In [40]:
fold.keys()

dict_keys(['sizes_train', 'components_train', 'num_patches_train', 'uniprots_train', 'labels_train', 'sizes_validation', 'components_validation', 'num_patches_validation', 'uniprots_validation', 'labels_validation', 'sizes_test', 'components_test', 'num_patches_test', 'uniprots_test', 'labels_test'])

In [50]:
fold[f'components_{set_name}'].shape

TensorShape([76632, 10, 9])

In [48]:
from typing import List, Dict, Union

def create_coordinates_matrix(uniprots: List[str], patches_ca_avg_coordinates: Dict[str, Union[np.ndarray, None]]) -> np.ndarray:
    data = [patches_ca_avg_coordinates[uniprot_id] for uniprot_id in uniprots]
    padded_data = []
    for item in data:
        if item is None:
            padded = np.zeros((10, 3))
        else:
            padded = np.vstack([item, np.zeros((10 - item.shape[0], 3))])
        padded_data.append(padded)
    return np.stack(padded_data)

In [60]:
for fold_index, fold in enumerate(folds_training_dicts):
    for set_name in ['train', 'validation', 'test']:
        uniprots = fold[f'uniprots_{set_name}']
        
        uniprots = [u for u in uniprots if u in all_patches_ca_coordinates]  # TODO: remove this line
        
        patches_ca_avg_coordinates = {uniprot_id: [np.average(c, axis=0) if c is not None else None for c in coordinates] for uniprot_id, coordinates in all_patches_ca_coordinates.items()}
        patches_ca_avg_coordinates = {k: np.vstack(v) if (len(v) > 0) else None for k, v in patches_ca_avg_coordinates.items()}
        coordinates_matrix = create_coordinates_matrix(uniprots, patches_ca_avg_coordinates)
        fold[f'coordinates_{set_name}'] = tf.convert_to_tensor(coordinates_matrix)

In [63]:
fold[f'coordinates_{set_name}']

<tf.Tensor: shape=(259, 10, 3), dtype=float64, numpy=
array([[[ -3.21264291,   6.66071463,  -7.96417856],
        [-16.50099945,   4.85699987,   2.78299999],
        [  0.        ,   0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]],

       [[ -4.85988951,  -8.35255527,  -1.78366685],
        [  3.27366662,  22.16516876,   7.96383286],
        [ 13.96800041,  14.87300014,  -6.98600006],
        ...,
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]],

       [[  4.1833787 ,  -2.94872427,  -8.45769024],
        [ 19.27820015, -10.96500111,   3.59220004],
        [ -9.88099957,   7.42533302,  16.19366646],
        ...,
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.      

In [34]:
# {k: v.shape for k, v in patches_ca_avg_coordinates.items() if v is not None}

In [35]:
# {k: v for k, v in patches_ca_avg_coordinates.items() if v is None}

In [36]:
# {k: v for k, v in data_set_res_indexes.items() if len(v) == 0}